<a href="https://colab.research.google.com/github/ahmadabousalem/sentinel-threat-map/blob/main/gazamap3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install feedparser folium pandas --quiet

import feedparser
import folium
import pandas as pd
import html
from IPython.display import IFrame, HTML, display
from datetime import datetime
import time

# ------------------ CONFIG ------------------
cities = {
    "Gaza": {"lat": 31.5017, "lon": 34.4668},
    "Beirut": {"lat": 33.8938, "lon": 35.5018},
    "Damascus": {"lat": 33.5138, "lon": 36.2765},
    "Northern Israel Border": {"lat": 33.27, "lon": 35.57}
}

# RSS feeds (free/open). If some are slow, remove while testing.
rss_feeds = [
    "https://www.aljazeera.com/xml/rss/all.xml",
    "http://feeds.bbci.co.uk/news/world/middle_east/rss.xml",
    "https://rss.dw.com/rdf/rss-en-all",
    "http://rss.cnn.com/rss/edition_world.rss",
    "http://feeds.reuters.com/Reuters/worldNews"
]

# Simple risk keywords
risk_keywords = [
    'attack', 'explosion', 'war', 'strike', 'missile', 'death',
    'protest', 'conflict', 'rocket', 'terror', 'bomb', 'hostage',
    'shooting', 'shelling', 'air raid'
]

# ------------------ HELPERS ------------------
def safe_parse(url):
    try:
        parsed = feedparser.parse(url)
        # basic check
        if parsed.bozo:
            # bozo flag means parse problem; still try to return entries if any
            return parsed
        return parsed
    except Exception as e:
        print(f"[WARN] Feed parse failed for {url}: {e}")
        return None

def fetch_risk_data(city_names, feeds, keywords, max_headlines=5):
    """
    Returns:
      counts: dict city -> integer score (sum keyword hits)
      headlines: dict city -> list of (pub, title, link)
    """
    counts = {c: 0 for c in city_names}
    headlines = {c: [] for c in city_names}
    for url in feeds:
        parsed = safe_parse(url)
        if not parsed or not hasattr(parsed, "entries"):
            continue
        for entry in parsed.entries:
            title = entry.get("title", "") or ""
            summary = entry.get("summary", "") or entry.get("description", "") or ""
            text = (title + " " + summary).lower()
            pub = entry.get("published", entry.get("updated", "N/A"))
            link = entry.get("link", "")
            for city in city_names:
                if city.lower() in text:
                    # count keyword matches (if none match, count as 1)
                    kcount = sum(1 for k in keywords if k in text)
                    counts[city] += (kcount if kcount>0 else 1)
                    # add headline (avoid duplicates)
                    entry_label = f"{pub} | {title}"
                    if entry_label not in [h[0] for h in headlines[city]]:
                        headlines[city].append((entry_label, link))
                        # keep only newest N headlines
                        headlines[city] = headlines[city][:max_headlines]
        # polite pause to avoid hammering feeds
        time.sleep(0.3)
    return counts, headlines

def classify_risk(score):
    if score >= 8:
        return "HIGH"
    elif score >= 4:
        return "MEDIUM"
    elif score > 0:
        return "LOW"
    else:
        return "NONE"

# ------------------ RUN FETCH ------------------
print("Fetching RSS feeds (may take 10-30s)...")
counts, headlines = fetch_risk_data(list(cities.keys()), rss_feeds, risk_keywords, max_headlines=5)
print("Done.\n")

# Make DataFrame for display
df = pd.DataFrame([{"city": c, "score": counts.get(c,0), "risk": classify_risk(counts.get(c,0))} for c in cities.keys()])
df = df.sort_values("score", ascending=False).reset_index(drop=True)
display(df)

# ------------------ BUILD MAP ------------------
m = folium.Map(location=[33.5, 35.4], zoom_start=7, tiles='CartoDB positron')

color_map = {"HIGH":"darkred", "MEDIUM":"orange", "LOW":"blue", "NONE":"green"}

for _, row in df.iterrows():
    city = row["city"]
    score = int(row["score"])
    level = row["risk"]
    coords = (cities[city]["lat"], cities[city]["lon"])
    color = color_map[level]
    radius = 6 + score * 2

    # Build popup HTML safely with links
    popup_html = f"<div style='max-width:350px'><b>{html.escape(city)}</b><br><b>Risk:</b> {level} (score {score})<br><hr>"
    if headlines.get(city):
        popup_html += "<b>Recent headlines:</b><br>"
        for h, link in headlines[city]:
            safe_h = html.escape(h)
            if link:
                popup_html += f"<a href='{link}' target='_blank'>{safe_h}</a><br>"
            else:
                popup_html += f"{safe_h}<br>"
    else:
        popup_html += "No recent headlines found.<br>"
    popup_html += "</div>"

    folium.CircleMarker(
        location=coords,
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=400),
        tooltip=f"{city}: {level} ({score})"
    ).add_to(m)

map_file = "sentinel_risk_map.html"
m.save(map_file)
print(f"Map saved to: {map_file}")

# Display map inline (two methods for robustness)
try:
    display(HTML(m._repr_html_()))
except Exception:
    display(IFrame(map_file, width="100%", height=600))
# ------------------ End of cell ------------------



Fetching RSS feeds (may take 10-30s)...
Done.



,city,score,risk
0,Gaza,41,HIGH
1,Beirut,0,NONE
2,Damascus,0,NONE
3,Northern Israel Border,0,NONE


Map saved to: sentinel_risk_map.html


In [ ]:

!pip install --quiet feedparser folium pandas requests python-dateutil

import feedparser, folium, pandas as pd, requests, time, html, os
from folium.plugins import MarkerCluster
from IPython.display import display, HTML
from datetime import datetime, timezone
from dateutil import parser as dateparser

# ---------------- CONFIG ----------------
CITY = "Gaza"
CENTER = (31.5017, 34.4668)

# Four professional POIs inside Gaza
LOCATIONS = {
    "Gaza City (Center)": {"lat": 31.5017, "lon": 34.4668, "aliases": ["gaza", "gaza city", "gaza strip"]},
    "Al-Shifa Hospital": {"lat": 31.5241, "lon": 34.4392, "aliases": ["shifa", "al-shifa", "al shifa", "shifa hospital"]},
    "Khan Yunis": {"lat": 31.3422, "lon": 34.3063, "aliases": ["khan yunis", "khan yunis"]},
    "Rafah Crossing": {"lat": 31.2833, "lon": 34.2486, "aliases": ["rafah", "rafah crossing", "rafah border"]}
}

RSS_FEEDS = [
    "https://www.aljazeera.com/xml/rss/all.xml",
    "http://feeds.bbci.co.uk/news/world/middle_east/rss.xml",
    "https://rss.dw.com/rdf/rss-en-all",
    "http://rss.cnn.com/rss/edition_world.rss",
    "http://feeds.reuters.com/Reuters/worldNews"
]

RISK_KEYWORDS = [
    'attack','explosion','war','strike','missile','death','injured','casualties',
    'protest','conflict','rocket','terror','bomb','hostage','shooting','shelling','air raid','evacuate','evacuation'
]

OUTFILE = "signals.csv"
MAX_HEADLINES_PER_LOC = 8

# ---------------- HELPERS ----------------
def safe_parse(url):
    try:
        return feedparser.parse(url)
    except Exception as e:
        print(f"[WARN] feed parse failed {url}: {e}")
        return None

def score_text(text):
    t = (text or "").lower()
    return sum(1 for kw in RISK_KEYWORDS if kw in t)

def risk_label(score):
    if score >= 8: return "CRITICAL"
    if score >= 5: return "HIGH"
    if score >= 2: return "MEDIUM"
    if score > 0:    return "LOW"
    return "NONE"

def risk_color(label):
    return {
        "CRITICAL": "#6d0000",
        "HIGH":     "#d9534f",
        "MEDIUM":   "#f0ad4e",
        "LOW":      "#5bc0de",
        "NONE":     "#5cb85c"
    }.get(label, "#777777")

def assign_location(text):
    lt = (text or "").lower()
    for name, info in LOCATIONS.items():
        for alias in info['aliases']:
            if alias.lower() in lt:
                return name
    return "Gaza City (Center)"

def find_thumbnail(entry):
    # RSS/Atom may carry media:content, media_thumbnail, or links with rel=enclosure
    # Try common keys safely
    try:
        if 'media_thumbnail' in entry and entry['media_thumbnail']:
            # sometimes a list
            thumb = entry['media_thumbnail'][0].get('url') if isinstance(entry['media_thumbnail'], list) else entry['media_thumbnail'].get('url')
            return thumb
        if 'media_content' in entry and entry['media_content']:
            thumb = entry['media_content'][0].get('url')
            return thumb
        # check links
        if 'links' in entry:
            for l in entry['links']:
                if l.get('rel') == 'enclosure' and l.get('type','').startswith('image'):
                    return l.get('href')
        # some feeds include 'image' or 'thumbnail' keys
        for k in ('image','thumbnail','pic'):
            if k in entry:
                val = entry[k]
                if isinstance(val, dict) and 'url' in val: return val['url']
                if isinstance(val, str): return val
    except Exception:
        pass
    return None

def save_signals(rows):
    write_header = not os.path.exists(OUTFILE)
    import csv
    with open(OUTFILE, "a", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        if write_header:
            w.writerow(["timestamp_utc","location","title","score","risk_label","source","published","link","thumbnail"])
        w.writerows(rows)

# ---------------- FETCH LIVE ARTICLES ----------------
matched = []
for feed in RSS_FEEDS:
    parsed = safe_parse(feed)
    if not parsed or not hasattr(parsed, 'entries'):
        continue
    source_name = getattr(parsed, 'feed', {}).get('title','') or feed
    for entry in parsed.entries:
        title = (entry.get('title') or "").strip()
        summary = (entry.get('summary') or entry.get('description') or "").strip()
        text = f"{title} {summary}"
        if CITY.lower() in text.lower() or any(alias in text.lower() for loc in LOCATIONS.values() for alias in loc['aliases']):
            score = score_text(text)
            label = risk_label(score)
            location = assign_location(text)
            link = entry.get('link','')
            published_raw = entry.get('published') or entry.get('updated') or ""
            # try to normalize published time for nicer display
            try:
                pub_dt = dateparser.parse(published_raw)
                published = pub_dt.strftime("%Y-%m-%d %H:%M:%S")
            except Exception:
                published = published_raw or ""
            thumb = find_thumbnail(entry)
            matched.append({
                "location": location,
                "title": title,
                "summary": summary,
                "source": source_name,
                "published": published,
                "link": link,
                "score": score,
                "risk": label,
                "thumbnail": thumb
            })
    time.sleep(0.2)

# If no live matches, add a friendly demo message rather than empty results
if len(matched) == 0:
    matched.append({
        "location": "Gaza City (Center)",
        "title": "No matched Gaza headlines right now",
        "summary": "Try again soon or add local Arabic feeds to increase coverage.",
        "source": "system",
        "published": datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC"),
        "link": "",
        "score": 0,
        "risk": "NONE",
        "thumbnail": None
    })

df = pd.DataFrame(matched)
# Save provenance rows to CSV
rows_to_save = []
ts = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")
for r in matched:
    rows_to_save.append([ts, r['location'], r['title'], r['score'], r['risk'], r['source'], r['published'], r['link'], r['thumbnail']])
save_signals(rows_to_save)

# ---------------- BUILD LUXURY FOLIUM MAP ----------------
m = folium.Map(location=CENTER, zoom_start=11, tiles="CartoDB positron")

# Top-left title card
title_html = f'''
<div style="position: fixed; top: 10px; left: 10px; z-index:9999;
            background: rgba(255,255,255,0.95); padding:12px; border-radius:8px;
            box-shadow: 2px 2px 8px rgba(0,0,0,0.12); font-family: Arial;">
  <div style="font-size:16px; font-weight:700;">SENTINEL • Gaza Intelligence</div>
  <div style="font-size:12px; margin-top:6px;">Live update: <b>{ts}</b></div>
  <div style="font-size:12px; margin-top:8px;">Articles matched: <b>{len(matched)}</b></div>
</div>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Add POI markers with attractive style and popup that contains top headlines for that POI
for loc_name, info in LOCATIONS.items():
    lat, lon = info['lat'], info['lon']
    loc_df = df[df['location'] == loc_name]
    total_score = int(loc_df['score'].sum()) if not loc_df.empty else 0
    risk_label_loc = risk_label(total_score)
    color = risk_color(risk_label_loc)
    popup_html = f"<div style='font-family: Arial; max-width:420px;'>"
    popup_html += f"<h4 style='margin:0 0 6px 0'>{html.escape(loc_name)}</h4>"
    popup_html += f"<div style='margin-bottom:6px'><b>Aggregated score:</b> {total_score} &nbsp; <b>Risk:</b> <span style='color:{color}'>{risk_label_loc}</span></div>"
    if not loc_df.empty:
        # Show up to MAX headlines in popup
        for _, row in loc_df.head(MAX_HEADLINES_PER_LOC).iterrows():
            title_short = html.escape(row['title'])[:120]
            src = html.escape(row['source'])
            pub = html.escape(row['published'])
            risk_b = row['risk']
            thumb = row['thumbnail']
            popup_html += "<div style='margin-bottom:8px; padding-bottom:6px; border-bottom:1px solid #eee;'>"
            if thumb:
                # include a tiny thumbnail left
                popup_html += f"<div style='display:flex; gap:8px; align-items:flex-start;'>"
                popup_html += f"<img src='{thumb}' style='width:88px; height:66px; object-fit:cover; border-radius:6px;'/>&nbsp;"
                popup_html += f"<div style='flex:1'><b style='font-size:13px'>{title_short}</b><br><small style='color:#666'>{src} • {pub}</small><br>"
                popup_html += f"<span style='color:{risk_color(risk_b)}; font-weight:700;'>{risk_b}</span> &nbsp; <a href='{row['link']}' target='_blank'>Read</a></div></div>"
            else:
                popup_html += f"<b style='font-size:13px'>{title_short}</b><br><small style='color:#666'>{src} • {pub}</small><br>"
                popup_html += f"<span style='color:{risk_color(risk_b)}; font-weight:700;'>{risk_b}</span> &nbsp; <a href='{row['link']}' target='_blank'>Read</a>"
            popup_html += "</div>"
    else:
        popup_html += "<i>No headlines for this location right now.</i>"
    popup_html += "</div>"

    folium.CircleMarker(
        location=(lat, lon),
        radius=12 + min(total_score, 12),
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.9,
        popup=folium.Popup(popup_html, max_width=520),
        tooltip=f"{loc_name} — {risk_label_loc} ({total_score})"
    ).add_to(m)

# Center marker with summary & quick actions (download CSV link)
summary_html = "<div style='font-family: Arial; max-width:520px;'>"
summary_html += f"<h4 style='margin:0 0 6px 0'>Gaza Dashboard Summary</h4>"
summary_html += f"<b>Run time:</b> {ts}<br><b>Total articles:</b> {len(matched)}<hr>"
if not df.empty:
    top = df.sort_values('score', ascending=False).head(6)
    for _, r in top.iterrows():
        summary_html += f"<div style='margin-bottom:8px'><b>{html.escape(r['title'])[:120]}</b><br><small style='color:#666'>{html.escape(r['source'])} • {html.escape(r['published'])}</small><br>"
        summary_html += f"<span style='color:{risk_color(r['risk'])}; font-weight:700'>{r['risk']}</span> &nbsp; <a href='{r['link']}' target='_blank'>Read</a></div>"
else:
    summary_html += "<i>No articles to show.</i>"
summary_html += "<hr><a href='/content/%s' target='_blank' download>Download signals.csv</a></div>" % OUTFILE

folium.Marker(
    location=CENTER,
    popup=folium.Popup(summary_html, max_width=560),
    icon=folium.Icon(color='darkblue', icon='info-sign')
).add_to(m)

# Display map inline (Colab-friendly)
display(HTML(m._repr_html_()))

# ---------------- NEWS BOARD (HTML) BELOW MAP ----------------
# Build a polished news board: grid of cards for each article (Gaza-wide)
news_cards = "<div style='font-family:Arial; margin-top:12px;'>"
news_cards += f"<h3>Live Gaza feed • {ts}</h3>"
news_cards += "<div style='display:flex; gap:12px; flex-wrap:wrap;'>"
for r in matched:
    thumb_html = f"<img src='{r['thumbnail']}' style='width:220px; height:140px; object-fit:cover; border-radius:6px;'/>" if r['thumbnail'] else "<div style='width:220px;height:140px;background:#f2f2f2;border-radius:6px;display:flex;align-items:center;justify-content:center;color:#777'>No image</div>"
    title_html = html.escape(r['title'])[:180]
    src = html.escape(r['source'])
    pub = html.escape(r['published'])
    risk = r['risk']
    color = risk_color(risk)
    card = f"""
      <div style='width:460px; border-radius:8px; background:white; box-shadow:0 4px 14px rgba(0,0,0,0.06); overflow:hidden;'>
        <div style='display:flex;'>
          <div style='flex:0 0 220px'>{thumb_html}</div>
          <div style='padding:12px; flex:1'>
            <div style='font-size:15px; font-weight:700; margin-bottom:6px'><a href='{r['link']}' target='_blank' style='color:#111; text-decoration:none'>{title_html}</a></div>
            <div style='font-size:12px; color:#666; margin-bottom:8px'>{src} • {pub}</div>
            <div style='margin-bottom:8px;'><b style="color:{color}; padding:3px 8px; border-radius:6px; background:rgba(0,0,0,0.03)">{risk}</b> &nbsp; Score: <b>{r['score']}</b></div>
            <div style='font-size:13px; color:#333'>{html.escape((r['summary'] or '')[:220])}{"..." if len((r['summary'] or ''))>220 else ""}</div>
          </div>
        </div>
      </div>
    """
    news_cards += card
news_cards += "</div></div>"

display(HTML(news_cards))

print("\nSaved run to signals.csv — you can download it from the Files pane or the dashboard link.")
# ----------------------------------------------------------------------------------------------
# End of cell: run again to refresh live data. If you want scheduled updates, I can show how to automate.
# ----------------------------------------------------------------------------------------------



Saved run to signals.csv — you can download it from the Files pane or the dashboard link.


In [ ]:


!pip install --quiet feedparser folium pandas requests python-dateutil

import feedparser, folium, pandas as pd, requests, time, html, os, base64, io
from folium.plugins import MarkerCluster
from IPython.display import display, HTML, clear_output
from datetime import datetime, timezone
from dateutil import parser as dateparser

# ---------------- CONFIG - tune these ----------------
CITY = "Gaza"
CENTER = (31.5017, 34.4668)
OUTFILE = "signals.csv"

AUTO_REFRESH = False           # Set True to auto-refresh until you stop the cell
REFRESH_INTERVAL_SECONDS = 60  # seconds between auto refresh runs

# POIs: 4 important points inside Gaza (you can adjust coordinates later)
LOCATIONS = {
    "Gaza City (Center)": {"lat": 31.5017, "lon": 34.4668, "aliases": ["gaza", "gaza city", "gaza strip"]},
    "Al-Shifa Hospital": {"lat": 31.5241, "lon": 34.4392, "aliases": ["shifa", "al-shifa", "al shifa", "shifa hospital"]},
    "Khan Yunis": {"lat": 31.3422, "lon": 34.3063, "aliases": ["khan yunis", "khan yunis"]},
    "Rafah Crossing": {"lat": 31.2833, "lon": 34.2486, "aliases": ["rafah", "rafah crossing", "rafah border", "rafah crossing"]}
}

# Feeds: add/remove if slow; local Arabic feeds can improve signal
RSS_FEEDS = [
    "https://www.aljazeera.com/xml/rss/all.xml",
    "http://feeds.bbci.co.uk/news/world/middle_east/rss.xml",
    "https://rss.dw.com/rdf/rss-en-all",
    "http://rss.cnn.com/rss/edition_world.rss",
    "http://feeds.reuters.com/Reuters/worldNews"
]

RISK_KEYWORDS = [
    'attack','explosion','war','strike','missile','death','injured','casualties',
    'protest','conflict','rocket','terror','bomb','hostage','shooting','shelling','air raid',
    'evacuate','evacuation','fired','clash','raid'
]

MAX_HEADLINES_PER_LOC = 8

# ---------------- helpers ----------------
def safe_parse(url):
    try:
        return feedparser.parse(url)
    except Exception as e:
        print(f"[WARN] feed parse failed {url}: {e}")
        return None

def score_text(text):
    t = (text or "").lower()
    return sum(1 for kw in RISK_KEYWORDS if kw in t)

def risk_label(score):
    if score >= 10: return "CRITICAL"
    if score >= 6: return "HIGH"
    if score >= 3: return "MEDIUM"
    if score > 0:    return "LOW"
    return "NONE"

def risk_color(label):
    return {
        "CRITICAL": "#6d0000",
        "HIGH":     "#d9534f",
        "MEDIUM":   "#f0ad4e",
        "LOW":      "#5bc0de",
        "NONE":     "#5cb85c"
    }.get(label, "#777777")

def assign_location(text):
    lt = (text or "").lower()
    for name, info in LOCATIONS.items():
        for alias in info['aliases']:
            if alias.lower() in lt:
                return name
    return "Gaza City (Center)"

def find_thumbnail(entry):
    try:
        if 'media_thumbnail' in entry and entry['media_thumbnail']:
            t = entry['media_thumbnail']
            if isinstance(t, list): t = t[0]
            return t.get('url') if isinstance(t, dict) else None
        if 'media_content' in entry and entry['media_content']:
            t = entry['media_content']
            if isinstance(t, list): t = t[0]
            return t.get('url')
        if 'links' in entry:
            for l in entry['links']:
                if l.get('rel') == 'enclosure' and l.get('type','').startswith('image'):
                    return l.get('href')
        for k in ('image','thumbnail','pic'):
            if k in entry:
                val = entry[k]
                if isinstance(val, dict) and 'url' in val: return val['url']
                if isinstance(val, str): return val
    except Exception:
        return None
    return None

def save_signals_csv(rows):
    write_header = not os.path.exists(OUTFILE)
    import csv
    with open(OUTFILE, "a", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        if write_header:
            w.writerow(["timestamp_utc","location","title","score","risk_label","source","published","link","thumbnail"])
        w.writerows(rows)

# Build the dashboard once (can be called repeatedly)
def build_dashboard():
    ts = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")
    matched = []
    # Fetch feeds
    for feed in RSS_FEEDS:
        parsed = safe_parse(feed)
        if not parsed or not hasattr(parsed, 'entries'):
            continue
        source_name = getattr(parsed, 'feed', {}).get('title','') or feed
        for entry in parsed.entries:
            title = (entry.get('title') or "").strip()
            summary = (entry.get('summary') or entry.get('description') or "").strip()
            text = f"{title} {summary}"
            # match if city or alias is found in article text
            if CITY.lower() in text.lower() or any(alias in text.lower() for loc in LOCATIONS.values() for alias in loc['aliases']):
                score = score_text(text)
                label = risk_label(score)
                location = assign_location(text)
                link = entry.get('link','')
                pub_raw = entry.get('published') or entry.get('updated') or ""
                try:
                    pub_dt = dateparser.parse(pub_raw)
                    published = pub_dt.strftime("%Y-%m-%d %H:%M:%S")
                except Exception:
                    published = pub_raw or ""
                thumb = find_thumbnail(entry) or ""
                matched.append({
                    "location": location,
                    "title": title,
                    "summary": summary,
                    "source": source_name,
                    "published": published,
                    "link": link,
                    "score": score,
                    "risk": label,
                    "thumbnail": thumb
                })
        time.sleep(0.2)

    # If nothing matched, add a demo message so UI isn't empty
    if len(matched) == 0:
        matched.append({
            "location": "Gaza City (Center)",
            "title": "No Gaza headlines matched right now",
            "summary": "Try running again or add more local feeds for better coverage.",
            "source": "system",
            "published": ts,
            "link": "",
            "score": 0,
            "risk": "NONE",
            "thumbnail": ""
        })

    df = pd.DataFrame(matched)

    # Persist to CSV
    rows = []
    for r in matched:
        rows.append([ts, r['location'], r['title'], r['score'], r['risk'], r['source'], r['published'], r['link'], r['thumbnail']])
    save_signals_csv(rows)

    # Aggregate per location
    agg_score = df.groupby("location")["score"].sum().to_dict()
    agg_count = df.groupby("location").size().to_dict()
    total_score = int(df["score"].sum()) if not df.empty else 0

    # Build Folium map
    m = folium.Map(location=CENTER, zoom_start=11, tiles="CartoDB positron")
    # Title card
    title_html = f'''
    <div style="position: fixed; top: 8px; left: 8px; z-index:9999;
                background: rgba(255,255,255,0.95); padding:10px; border-radius:8px;
                box-shadow: 2px 2px 10px rgba(0,0,0,0.12); font-family: Arial;">
      <div style="font-size:15px; font-weight:700;">SENTINEL — Gaza Intelligence</div>
      <div style="font-size:12px; margin-top:6px">Updated: <b>{ts}</b></div>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(title_html))

    # Add POI markers
    for loc_name, info in LOCATIONS.items():
        lat, lon = info['lat'], info['lon']
        loc_df = df[df["location"] == loc_name]
        loc_score = int(agg_score.get(loc_name, 0))
        loc_count = int(agg_count.get(loc_name, 0))
        loc_label = risk_label(loc_score)
        color = risk_color(loc_label)
        # build popup HTML for the location
        popup_html = f"<div style='font-family:Arial; max-width:420px'><h4 style='margin:0'>{html.escape(loc_name)}</h4>"
        popup_html += f"<div style='margin:6px 0'><b>Aggregated:</b> {loc_score} • <b>Count:</b> {loc_count} • <b>Risk:</b> <span style='color:{color}'>{loc_label}</span></div>"
        if not loc_df.empty:
            for _, r in loc_df.head(MAX_HEADLINES_PER_LOC).iterrows():
                title_s = html.escape(r["title"])[:120]
                src = html.escape(r["source"])
                pub = html.escape(r["published"])
                popup_html += "<div style='margin-bottom:8px; padding-bottom:6px; border-bottom:1px solid #eee;'>"
                if r["thumbnail"]:
                    popup_html += f"<div style='display:flex; gap:8px;'><img src='{r['thumbnail']}' style='width:90px;height:66px;object-fit:cover;border-radius:6px;'/><div>"
                    popup_html += f"<b>{title_s}</b><br><small style='color:#666'>{src} • {pub}</small><br>"
                    popup_html += f"<b style='color:{risk_color(r['risk'])}'>{r['risk']}</b> • <a href='{r['link']}' target='_blank'>Read</a></div></div>"
                else:
                    popup_html += f"<b>{title_s}</b><br><small style='color:#666'>{src} • {pub}</small><br>"
                    popup_html += f"<b style='color:{risk_color(r['risk'])}'>{r['risk']}</b> • <a href='{r['link']}' target='_blank'>Read</a>"
                popup_html += "</div>"
        else:
            popup_html += "<i>No recent headlines.</i>"
        popup_html += "</div>"

        folium.CircleMarker(
            location=(lat, lon),
            radius=12 + min(loc_score, 12),
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.9,
            popup=folium.Popup(popup_html, max_width=520),
            tooltip=f"{loc_name}: {loc_label} ({loc_score})"
        ).add_to(m)

    # center summary marker
    summary_html = "<div style='font-family:Arial; max-width:520px'>"
    summary_html += f"<h4 style='margin:0'>Gaza Summary</h4><div style='margin:6px 0'><b>Total score:</b> {total_score} • <b>Articles:</b> {len(df)}</div>"
    # top articles
    if not df.empty:
        top = df.sort_values("score", ascending=False).head(6)
        for _, r in top.iterrows():
            summary_html += f"<div style='margin-bottom:8px'><b>{html.escape(r['title'])[:120]}</b><br><small style='color:#666'>{html.escape(r['source'])} • {html.escape(r['published'])}</small><br>"
            summary_html += f"<span style='color:{risk_color(r['risk'])}'>{r['risk']}</span> • <a href='{r['link']}' target='_blank'>Read</a></div>"
    summary_html += f"<hr><a href='/content/{OUTFILE}' target='_blank' download>Download signals.csv</a></div>"

    folium.Marker(
        location=CENTER,
        popup=folium.Popup(summary_html, max_width=560),
        icon=folium.Icon(color='darkblue', icon='info-sign')
    ).add_to(m)

    # Build side panel: gauge + ticker + cards as an HTML string
    # Gauge: we use total_score mapped to 0-100 for visuals
    gauge_value = min(100, total_score * 4)  # scale approx
    agg_label_overall = risk_label(total_score)

    # ticker items
    ticker_html = ""
    for _, r in df.sort_values("score", ascending=False).head(12).iterrows():
        ticker_html += f"<span style='margin-right:36px; color:{risk_color(r['risk'])};'><b>[{r['risk']}]</b> {html.escape(r['title'])[:120]}</span>"

    # cards (compact) for feed below the map
    cards_html = ""
    for _, r in df.sort_values("score", ascending=False).iterrows():
        thumb = f"<img src='{r['thumbnail']}' style='width:120px;height:80px;object-fit:cover;border-radius:6px'/>" if r['thumbnail'] else "<div style='width:120px;height:80px;background:#f2f2f2;border-radius:6px;display:flex;align-items:center;justify-content:center;color:#888'>No image</div>"
        cards_html += f"""
        <div style='display:flex; gap:10px; padding:10px; border-radius:8px; background:white; box-shadow:0 4px 12px rgba(0,0,0,0.06); margin-bottom:12px;'>
          <div>{thumb}</div>
          <div style='flex:1'>
            <div style='font-weight:700; font-size:14px; margin-bottom:6px'><a href='{r['link']}' target='_blank' style='color:#111;text-decoration:none'>{html.escape(r['title'])[:140]}</a></div>
            <div style='font-size:12px; color:#666; margin-bottom:8px'>{html.escape(r['source'])} • {html.escape(r['published'])}</div>
            <div><span style='color:{risk_color(r['risk'])}; font-weight:700'>{r['risk']}</span> &nbsp; Score: <b>{r['score']}</b></div>
            <div style='font-size:13px; margin-top:6px; color:#333'>{html.escape((r['summary'] or '')[:200])}{'...' if len((r['summary'] or ''))>200 else ''}</div>
          </div>
        </div>
        """

    # Compose full HTML (two-column layout)
    map_html = m._repr_html_()
    side_html = f"""
    <div style='font-family: Arial; display:flex; flex-direction:column; gap:12px;'>
      <div style='width:420px; background:linear-gradient(180deg,#ffffff,#f8f8f8); padding:12px; border-radius:8px; box-shadow:0 6px 20px rgba(0,0,0,0.08)'>
        <div style='font-size:18px; font-weight:700'>Threat Panel — Gaza</div>
        <div style='margin-top:8px; font-size:13px'>Overall: <b style='color:{risk_color(agg_label_overall)}'>{agg_label_overall}</b> • Score: <b>{total_score}</b></div>
        <div style='margin-top:10px; display:flex; align-items:center; gap:12px'>
          <div style='width:120px; height:120px; border-radius:60px; background:linear-gradient(180deg,#fff,#eee); display:flex; align-items:center; justify-content:center; font-size:24px; font-weight:800; color:{risk_color(agg_label_overall)}'>
            {agg_label_overall[:3]}
          </div>
          <div style='flex:1'>
            <div style='margin-bottom:6px'><button id="refresh_now" style="padding:8px 12px; background:#2b7be4;color:white;border:none;border-radius:6px;cursor:pointer">Refresh now</button>
            <button id="toggle_auto" style="padding:8px 12px; margin-left:6px; background:#6c757d;color:white;border:none;border-radius:6px;cursor:pointer">Auto: {str(AUTO_REFRESH)}</button></div>
            <div style='font-size:13px; color:#555'>Latest headlines: </div>
            <div style='margin-top:8px; color:#444; overflow:hidden; white-space:nowrap;' id='ticker'>{ticker_html}</div>
          </div>
        </div>
      </div>
      <div style='width:420px; max-height:560px; overflow:auto;'>
        {cards_html}
      </div>
    </div>
    """

    full_html = f"""
    <div style='display:flex; gap:18px;'>
      <div style='flex:1; min-width:650px;'>{map_html}</div>
      <div style='width:460px'>{side_html}</div>
    </div>
    <script>
    // Simple JS to wire Refresh button back to Colab: it will just focus the output cell so you can re-run
    const btn = document.getElementById('refresh_now');
    const toggle = document.getElementById('toggle_auto');
    if(btn) btn.onclick = ()=>{{ alert('To refresh: re-run this cell (press Ctrl+Enter). Auto-refresh is experimental in Colab.') }};
    if(toggle) toggle.onclick = ()=>{{ alert('Auto-refresh must be set in the Python variable AUTO_REFRESH and re-run.'); }};
    </script>
    """
    display(HTML(full_html))
    print(f"Built dashboard — {ts} — matched {len(df)} articles — signals saved to {OUTFILE}")

# Run once or loop if AUTO_REFRESH True
if AUTO_REFRESH:
    try:
        while True:
            clear_output(wait=True)
            build_dashboard()
            time.sleep(REFRESH_INTERVAL_SECONDS)
    except KeyboardInterrupt:
        print("Auto-refresh stopped.")
else:
    build_dashboard()

# ---------------------------------------------------------------------------------------
# Tips:

Built dashboard — 2025-08-12 18:07:39 UTC — matched 37 articles — signals saved to signals.csv
